## Import Libraries

In [15]:
import numpy as np


import time
import tensorflow as tf
from tensorflow.keras import optimizers
from sklearn import metrics

from utils.utils import *
from models.gcn import GCN

from config import CONFIG
import os

## Set genral parameters

In [10]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
cfg = CONFIG()

# set random seed
seed = 6606
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
dataset = 'R8'
cfg.dataset = dataset  

## Load data

In [4]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size = load_corpus(cfg.dataset)

features = sp.identity(features.shape[0])  # featureless
features = preprocess_features(features)

Generate features, labels, etc.

In [5]:
support = [preprocess_adj(adj)]


t_features = tf.SparseTensor(*features)
t_y_train = tf.convert_to_tensor(y_train)
t_y_val = tf.convert_to_tensor(y_val)
t_y_test = tf.convert_to_tensor(y_test)
tm_train_mask = tf.convert_to_tensor(train_mask)

tm_val_mask = tf.convert_to_tensor(val_mask)
tm_test_mask = tf.convert_to_tensor(test_mask)

t_support = []
for i in range(len(support)):
    t_support.append(tf.cast(tf.SparseTensor(*support[i]), dtype=tf.float64))

## Load model

In [6]:
model = GCN(input_dim=features[2][1], output_dim=y_train.shape[1], num_features_nonzero=features[1].shape)

input  dim:  15362
output dim:  8
Instructions for updating:
Please use `layer.add_weight` method instead.


Set activation function

In [7]:
optimizer = optimizers.Adam(lr=cfg.learning_rate)

## Train

In [11]:
cost_val = []
for epoch in range(cfg.epochs):
    
    t = time.time()
    with tf.GradientTape() as tape:
        _, loss, acc = model((t_features, t_y_train, tm_train_mask, t_support))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    _, val_loss, val_acc = model((t_features, t_y_val, tm_val_mask, t_support), training=False)
    cost_val.append(val_loss)
    
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(loss),
          "train_acc=", "{:.5f}".format(acc), "val_loss=", "{:.5f}".format(val_loss),
          "val_acc=", "{:.5f}".format(val_acc), "time=", "{:.5f}".format(time.time() - t))
    
    if epoch > cfg.early_stopping and cost_val[-1] > np.mean(cost_val[-(cfg.early_stopping+1):-1]):
        print("Early stopping...")
        break

Epoch: 0001 train_loss= 0.06394 train_acc= 0.98339 val_loss= 0.10886 val_acc= 0.95985 time= 0.62827
Epoch: 0002 train_loss= 0.06571 train_acc= 0.98137 val_loss= 0.10808 val_acc= 0.95985 time= 0.62754
Epoch: 0003 train_loss= 0.06388 train_acc= 0.98218 val_loss= 0.10799 val_acc= 0.96168 time= 0.62758
Epoch: 0004 train_loss= 0.06270 train_acc= 0.98440 val_loss= 0.10890 val_acc= 0.95985 time= 0.63014
Epoch: 0005 train_loss= 0.05714 train_acc= 0.98683 val_loss= 0.11019 val_acc= 0.95985 time= 0.63138
Epoch: 0006 train_loss= 0.06074 train_acc= 0.98481 val_loss= 0.11198 val_acc= 0.95803 time= 0.62789
Epoch: 0007 train_loss= 0.05852 train_acc= 0.98562 val_loss= 0.11471 val_acc= 0.95438 time= 0.62943
Epoch: 0008 train_loss= 0.05659 train_acc= 0.98744 val_loss= 0.11766 val_acc= 0.94891 time= 0.63082
Epoch: 0009 train_loss= 0.05519 train_acc= 0.98562 val_loss= 0.11946 val_acc= 0.94891 time= 0.62694
Epoch: 0010 train_loss= 0.05668 train_acc= 0.98683 val_loss= 0.11915 val_acc= 0.94891 time= 0.62909


## Evaluate

In [17]:
def evaluate(features, y, mask, support):
    t = time.time()
    
    pred, test_loss, test_acc = model((features, y, mask, support), training=False)
    
    
    return test_loss, test_acc, pred, np.argmax(y, axis=1), time.time() - t

In [18]:
test_cost, test_acc, pred, labels, test_duration = evaluate(t_features, t_y_test, tm_test_mask, t_support)
print("Test set results:", "cost=", "{:.5f}".format(test_cost), "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))


test_pred = []
test_labels = []

for i in range(len(test_mask)):
    if test_mask[i]:
        test_pred.append(pred[i])
        test_labels.append(labels[i])

print("Average Test Precision, Recall and F1-Score...")
print(metrics.precision_recall_fscore_support(test_labels, test_pred, average='micro'))

Test set results: cost= 0.11257 accuracy= 0.97213 time= 0.11193
Average Test Precision, Recall and F1-Score...
(0.972133394243947, 0.972133394243947, 0.972133394243947, None)
